In [ ]:
using Pkg
Pkg.activate("../../environments/refined-delaunay-for-flow-problems/");

In [ ]:
using LinearAlgebra
using Random
using SparseArrays
using Statistics

using AbstractPlotting
using CairoMakie
using Distances
using JuMP
using LightGraphs
using Parameters
using SCIP
using TriangleMesh

# Random Instance Data

In [ ]:
# create instance data
Random.seed!(0);

const N = 7
const WIDTH = 500
const HEIGHT = 300

x = round.(0.95 * WIDTH * rand(N))
y = round.(0.95 * HEIGHT * rand(N))
points = [x y]

# Triangulation with TriangleMesh

In [ ]:
@with_kw struct TriangleSwitches
    minimum_angle = nothing           # numeric
    maximum_area = nothing            # numeric
    conforming_delaunay = false
    maximum_steiner_points = nothing  # numeric
    no_boundary_steiner_points = false
    check_consistency = true
    quiet = true
end

function Base.convert(String, x::TriangleSwitches)
    s = ""
    s *= "e" # always output edges
    if x.minimum_angle !== nothing
        s *= "q$(x.minimum_angle)"
    end
    if x.maximum_area !== nothing
        s *= "a$(x.maximum_area)"
    end
    if x.conforming_delaunay
        s *= "D"
    end
    if x.maximum_steiner_points !== nothing
        s *= "S$(x.maximum_steiner_points)"
    end
    if x.no_boundary_steiner_points
        s *= "Y"
    end
    if x.check_consistency
        s *= "C"
    end
    if x.quiet
        s *= "Q"
    end
    return s
end

In [ ]:
"Create polygon from points and edges (used as segments)."
function Polygon(points, edges)
    n_point = size(points, 1)
    n_point_marker = 0
    n_point_attribute = 0
    n_segment = size(edges, 1)
    n_hole = 0
    poly = Polygon_pslg(n_point, n_point_marker, n_point_attribute, n_segment, n_hole)
    set_polygon_point!(poly, points)
    set_polygon_segment!(poly, edges)
    return poly
end

Polygon(trimesh::TriMesh) = Polygon(trimesh.point', trimesh.edge')

In [ ]:
function triangulate(points::Matrix{Float64}, switches::TriangleSwitches)::TriMesh
    s = "c" # create mesh from point Cloud
    s *= convert(String, switches)
    return create_mesh(points, s)
end

function triangulate(poly::Polygon_pslg, switches::TriangleSwitches)::TriMesh
    s = "p" # create mesh from Polygon
    s *= convert(String, switches)
    return create_mesh(poly, s)
end

In [ ]:
function delaunay_triangulation(points::Matrix{Float64})::TriMesh
    switches = TriangleSwitches(conforming_delaunay=true, maximum_steiner_points=0)
    return triangulate(points, switches)
end

In [ ]:
deltri = delaunay_triangulation(points);

In [ ]:
delpoly = Polygon(deltri);

# Drawing with Makie

In [ ]:
function make_scene(width=WIDTH, height=HEIGHT)
    return Scene(resolution=(width, height), show_axis=false, scale_plot=false)
end

function draw_points!(points; markersize=4, color=:black)
    scatter!(points[:, 1], points[:, 2], markersize=markersize, color=color)
end

function draw_edges!(points, edges; color=:gray)
    linesegments!(points[edges'[:], :], color=color)
end

function draw_triangulation(points, edges)
    make_scene()
    draw_edges!(points, edges)
    draw_points!(points)   
end

draw_triangulation(trimesh::TriMesh) = draw_triangulation(trimesh.point', trimesh.edge')

In [ ]:
draw_triangulation(deltri)

In [ ]:
draw_triangulation(triangulate(points,
        TriangleSwitches(minimum_angle=20, conforming_delaunay=true)))

In [ ]:
draw_triangulation(triangulate(delpoly,
        TriangleSwitches(minimum_angle=20, conforming_delaunay=true)))

# Triangle Centers

In [ ]:
pointset_mean(array) = dropdims(mean(array, dims=2), dims=2)

In [ ]:
abstract type TriangleCenter end
struct TriangleCentroid <: TriangleCenter end
struct TriangleIncenter <: TriangleCenter end
struct TriangleCircumcenter <: TriangleCenter end # yield Voronoi points!

function triangle_centers(trimesh, ::TriangleCentroid)
    @unpack point, cell = trimesh
    return pointset_mean(point'[cell', :])
end

function triangle_centers(trimesh, ::TriangleIncenter)
    @unpack point, cell = trimesh
    centers = []
    for t in eachrow(cell')
        corners = point'[t, :]

        a = norm(corners[2, :] - corners[3, :])
        b = norm(corners[1, :] - corners[3, :])
        c = norm(corners[1, :] - corners[2, :])
        # based on barycentric coordinates a:b:c        
        incenter = [a b c] * corners ./ (a + b + c)
        push!(centers, incenter)
    end
    
    return vcat(centers...)
end

function triangle_centers(trimesh, ::TriangleCircumcenter)
    @unpack point, cell = trimesh
    centers = []
    for t in eachrow(cell')
        corners = point'[t, :]
        Ax, Ay = corners[1, :]
        Bx, By = corners[2, :]
        Cx, Cy = corners[3, :]
        D = 2 * ( Ax * (By - Cy) + Bx * (Cy - Ay) + Cx * (Ay - By) )
        Ux = ((Ax^2 + Ay^2)*(By - Cy)) + ((Bx^2 + By^2)*(Cy - Ay)) + ((Cx^2 + Cy^2)*(Ay - By))
        Uy = ((Ax^2 + Ay^2)*(Cx - Bx)) + ((Bx^2 + By^2)*(Ax - Cx)) + ((Cx^2 + Cy^2)*(Bx - Ax))
        circumcenter = [Ux Uy] ./ D
        push!(centers, circumcenter)
    end
    
    return vcat(centers...)
end

In [ ]:
draw_triangulation(deltri)
draw_points!(triangle_centers(deltri, TriangleCentroid()), color=:orange)
draw_points!(triangle_centers(deltri, TriangleIncenter()), color=:red)
draw_points!(triangle_centers(deltri, TriangleCircumcenter()), color=:blue)

# Steiner Tree Model

In [ ]:
function antiparallel_digraph(trimesh)
    points, edges = trimesh.point', trimesh.edge'
    graph = SimpleDiGraph(size(points, 1))
    for e in 1:size(edges, 1)
        s, t = edges[e, :]
        add_edge!(graph, s, t)
        add_edge!(graph, t, s)
    end
    return graph
end

function edge_lengths(points, edges)
    diff = points[edges[:, 1], :] - points[edges[:, 2], :]
    return dropdims(mapslices(norm, diff, dims=2), dims=2)
end

edge_lengths(trimesh) = edge_lengths(trimesh.point', trimesh.edge')

function edge_length_map(trimesh)
    edges = trimesh.edge'
    lengths = edge_lengths(trimesh)

    length_map = Dict{Tuple{Int64, Int64}, Float64}()
    for e = 1:size(edges, 1)
        s, t = edges[e, :]
        length_map[s, t] = lengths[e]
        length_map[t, s] = lengths[e]
    end
    return length_map
end

function steiner_tree(trimesh, terminals)
    # Compute length by Euclidean distance of nodes.
    lengths = edge_length_map(trimesh)
    
    # Build digraph with all antiparallel arcs, for nonnegative flow.
    graph = antiparallel_digraph(trimesh)
    nodes = collect(1:nv(graph))
    arcs = collect(keys(lengths))
    
    length(terminals) >= 2 || error("Need at least 2 terminals.")
    all(terminals .<= nv(graph)) || error("Terminals out of range.")
    root = terminals[1]
    sinks = terminals[2:end]
   
    demand(v, s) = 1.0*(v == s) - 1.0*(v == root)
    
    # Using arc length for fixed capacity cost and multi-commodity flow.
    model = JuMP.direct_model(SCIP.Optimizer(display_verblevel=0))
    @variable(model, select[a in arcs], Bin, container=SparseAxisArray)
    @variable(model, flow[a in arcs,s in sinks] ≥ 0, container=SparseAxisArray)
    @constraint(model, balance[v in nodes, s in sinks],
        sum(flow[(n, v), s] - flow[(v, n), s] for n in neighbors(graph, v))
        == demand(v, s))
    @constraint(model, capacity[a in arcs, s in sinks], flow[a, s] <= select[a])
    @objective(model, Min, sum(lengths[a] * select[a] for a in arcs))
    
    optimize!(model)
    
    return objective_value(model), value.(select)
end

In [ ]:
function draw_tree(trimesh, terminals=1:N)
    points, edges = trimesh.point', trimesh.edge'
    obj, select = steiner_tree(trimesh, terminals)
    @show obj
    selected = [select[(s,t)] + select[(t,s)] for (s,t) in eachrow(edges)]
    active_edges = edges[selected .> 0.0, :]
    
    draw_triangulation(trimesh)
    linesegments!(points[active_edges'[:], :], color=:plum, linewidth=3)
    draw_points!(points[terminals, :], markersize=5, color=:teal)
end

In [ ]:
draw_tree(deltri)

In [ ]:
tri = triangulate(points, TriangleSwitches(minimum_angle=30, conforming_delaunay=true))
draw_tree(tri)

In [ ]:
tri = triangulate(points, TriangleSwitches(maximum_area=sqrt(WIDTH*HEIGHT), conforming_delaunay=true))
draw_tree(tri)

In [ ]:
tri = triangulate(delpoly, TriangleSwitches(maximum_area=sqrt(WIDTH*HEIGHT), conforming_delaunay=true))
draw_tree(tri)

In [ ]:
tri = triangulate(points, TriangleSwitches(maximum_area=sqrt(WIDTH*HEIGHT), conforming_delaunay=true,
                                           no_boundary_steiner_points=true))
draw_tree(tri)

# Distances and Shortest Paths

In [ ]:
function graph(trimesh)
    points, edges = trimesh.point', trimesh.edge'
    graph = SimpleGraph(size(points, 1))
    for e in 1:size(edges, 1)
        s, t = edges[e, :]
        add_edge!(graph, s, t)
    end
    return graph
end

function distance_matrix(trimesh)
    points = trimesh.point'
    return pairwise(Euclidean(), points, points, dims=1)
end

"Shortest path distance between two opposing vertices of adjacent triangles."
function shortest_path(graph, s, t, distances)
    heur(n) = distances[n, t]
    path = a_star(graph, s, t, distances, heur)
    return sum(distances[src(edge), dst(edge)] for edge in path)
end